# Final Report_v2

* Author : 楊右宇  
* Establish Date：2021.01.06  
* Last Modified Date：2021.01.06

# import library

In [46]:
import pandas as pd
import numpy as np
import math
import mplfinance as mpf

# Indicators

## Moving Average

In [47]:
def MA(_ticker, n_days=10, field='Close'):
    """ make the new colume names as <ma_@n_days>, the value 
        will be the @n_days average of the @Close price"""

    res = _ticker.copy(deep=True)
    if field == 'Volume':
        res['VMA_' + str(n_days)] = _ticker[field].rolling(n_days).mean()

    else:
        res['MA_' + str(n_days)] = _ticker[field].rolling(n_days).mean()

    return res

## EMA

In [48]:
def EMA(_ticker, period=15):
    '''
        計算EMA，並新增在 DataFrame 中'''
    # 複製一份資料，避免修改原始資料
    res = _ticker.copy(deep=True)
    # 新增新的一欄在 DataFrame 中
    res['EMA_' + str(period)] = _ticker['Close'].ewm(span=period, adjust=False).mean()

    return res

## RSI

* Relative Strength Index
* 相對強弱指標
* 計算出一個0~100之間的數字
* 越接近０表示盤勢越弱，越容易下跌
* 越接近100表示盤勢越強，越容易上漲

In [49]:
def RSI(_ticker, n_days = 14):
    
    # Create a empty dataFrame with original index
    # To store some temporary data
    tmp = pd.DataFrame(index=_ticker.index)

    # Copy a dataFrame to store result
    # deep=True means make a copy instead of link
    res = _ticker.copy(deep=True)

    tmp['rise'] = np.nan

    col_rise = tmp.columns.get_loc('rise')
    col_close = _ticker.columns.get_loc('Close')

    for i in range(1, len(_ticker)):
        tmp.iloc[i, col_rise] = (_ticker.iloc[i, col_close] - _ticker.iloc[i - 1, col_close])


    tmp['rise_only'] = tmp['rise']
    tmp['drop_only'] = tmp['rise']

    tmp['rise_only'].loc[tmp['rise'] < 0] = 0
    tmp['drop_only'].loc[tmp['rise'] > 0] = 0
    
    tmp['rise_avg'] = tmp['rise_only'].rolling(n_days).mean()
    tmp['drop_avg'] = tmp['drop_only'].rolling(n_days).mean().abs()

    res['RSI'] = tmp['rise_avg'] / (tmp['rise_avg'] + tmp['drop_avg']) * 100
    
    return res

In [50]:
### 410974045 ###
def RSI_2(_ticker,n_days = 14):
    
    #複製原RSI過來的（ 存入時 改成有分 是多少日的RSI ）
    # Create a empty dataFrame with original index
    # To store some temporary data
    tmp = pd.DataFrame(index=_ticker.index)

    # Copy a dataFrame to store result
    # deep=True means make a copy instead of link
    res = _ticker.copy(deep=True)

    tmp['rise'] = np.nan

    col_rise = tmp.columns.get_loc('rise')
    col_close = _ticker.columns.get_loc('Close')

    for i in range(1, len(_ticker)):
        tmp.iloc[i, col_rise] = (_ticker.iloc[i, col_close] - _ticker.iloc[i - 1, col_close])


    tmp['rise_only'] = tmp['rise']
    tmp['drop_only'] = tmp['rise']

    tmp['rise_only'].loc[tmp['rise'] < 0] = 0
    tmp['drop_only'].loc[tmp['rise'] > 0] = 0
    
    tmp['rise_avg'] = tmp['rise_only'].rolling(n_days).mean()
    tmp['drop_avg'] = tmp['drop_only'].rolling(n_days).mean().abs()

    res['RSI_' + str(n_days)] = tmp['rise_avg'] / (tmp['rise_avg'] + tmp['drop_avg']) * 100
    
    return res

## KD

* Ｋ值比Ｄ值敏感，波動幅度較大
* Ｋ值線如果在低檔區由下而上穿過Ｄ值線，稱作黃金交叉，代表接下來盤勢上漲機會大
* Ｋ值線如果在高檔區由上而下穿過Ｄ值線，稱作死亡交叉，代表接下來盤勢下跌機會大

In [51]:
def KD(_ticker, n_days = 9):

    # Create a empty dataFrame with original index
    # To store some temporary data
    tmp = pd.DataFrame(index=_ticker.index)

    # Copy a dataFrame to store result
    # deep=True means make a copy instead of link
    res = _ticker.copy(deep=True)

    tmp['max_close'] = ticker['Close'].rolling(n_days).max()
    tmp['min_close'] = ticker['Close'].rolling(n_days).min()
    # tmp['max_close'] = ticker['Close'].rolling(n_days, min_periods=1).max()
    # tmp['min_close'] = ticker['Close'].rolling(n_days, min_periods=1).min()

    # RSV also called FastK
    tmp['RSV'] = (ticker['Close'] - tmp['min_close']) / (tmp['max_close'] - tmp['min_close']) * 100

    res['K'] = 50
    res['D'] = 50
    
    col_k = res.columns.get_loc('K')
    col_d = res.columns.get_loc('D')
    col_rsv = tmp.columns.get_loc('RSV')

    # for i in range(1, len(_ticker)):
    for i in range(9, len(_ticker)):
        res.iloc[i, col_k] = res.iloc[i - 1, col_k] * 2 / 3 + tmp.iloc[i, col_rsv] * 1 / 3
        res.iloc[i, col_d] = res.iloc[i - 1, col_d] * 2 / 3 + res.iloc[i, col_k] * 1 / 3
    
    return res

## MACD

* macd（對應diff）
* macdsignal（對應dea）
* macdhist（對應macd）

### 簡易策略
* DIFF、DEA均為正，DIFF向上突破DEA，買入信號。

* DIFF、DEA均為負，DIFF向下跌破DEA，賣出信號。

* DEA線與K線發生背離，行情反轉信號。

* 分析MACD柱狀線，由正變負，賣出信號；由負變正，買入信號。

In [52]:
def MACD(_ticker, _fastperiod=12, _slowperiod=26, _signalperiod=9):
    
    # 複製一份資料，避免修改原始資料
    res = _ticker.copy(deep=True)

    ema_f = _ticker['Close'].ewm(span=_fastperiod, adjust=False).mean()
    ema_s = _ticker['Close'].ewm(span=_slowperiod, adjust=False).mean()
    res['macd_dif'] = ema_f - ema_s
    res['macd_dea'] = res['macd_dif'].ewm(span=_signalperiod, adjust=False).mean()
    res['macd'] = res['macd_dif'] - res['macd_dea']

    return res

## Bollinger Bands

In [53]:
def BBands(_ticker, period=20, std_num=2):
    
    # 複製一份資料，避免修改原始資料
    res = _ticker.copy(deep=True)

    # 計算中線，也就是二十日MA
    res['bbands_middle'] = _ticker['Close'].rolling(period).mean()
    
    # Calculate the Standard Deviation
    # 計算標準差
    std = _ticker['Close'].rolling(period).std(ddof=0) 
    
    # Calculate top band
    res['bbands_upper'] = res['bbands_middle'] + std * std_num 
    # Calculate bottom band
    res['bbands_lower'] = res['bbands_middle'] - std * std_num

    return res

# 交易策略

In [54]:
### 410974045 ###
def kd_1_strategy(_ticker):
    
    ## 在單一策略裡呼叫需要使用的KD
    _ticker = KD(_ticker, 9)
    
    res = _ticker.copy(deep=True)
    res['buy_point'] = 0
    col_b = res.columns.get_loc('buy_point')
    
    ## 9日K交叉9日D向上時買進 \ 9日K交叉9日D向下時賣出
    for i in range(1,len(_ticker)):
        if _ticker.loc[_ticker.index[i-1],'K'] < _ticker.loc[_ticker.index[i-1], 'D'] \
            and _ticker.loc[_ticker.index[i],'K'] > _ticker.loc[_ticker.index[i], 'D']:
                res.iloc[i, col_b] = 1
                
        elif _ticker.loc[_ticker.index[i-1],'K'] > _ticker.loc[_ticker.index[i-1], 'D'] \
            and _ticker.loc[_ticker.index[i],'K'] < _ticker.loc[_ticker.index[i], 'D']:
                res.iloc[i, col_b] = -1

    return res

In [55]:
### 410974045 ###
def rsi_1_strategy(_ticker):
    
    ## 在單一策略裡呼叫需要使用的RSI
    _ticker = RSI_2(_ticker, 6)  
    _ticker = RSI_2(_ticker, 12)
    
    res = _ticker.copy(deep=True)
    res['buy_point'] = 0
    col_b = res.columns.get_loc('buy_point')
    
    ## 6日RSI交叉12日RSI向上時買進 \ 6日RSI交叉12日RSI向下時賣出
    for i in range(1,len(_ticker)):
        if _ticker.loc[_ticker.index[i-1],'RSI_6'] < _ticker.loc[_ticker.index[i-1], 'RSI_12'] \
            and  _ticker.loc[_ticker.index[i],'RSI_6'] > _ticker.loc[_ticker.index[i], 'RSI_12']:
                res.iloc[i, col_b] = 1
                
        elif _ticker.loc[_ticker.index[i-1],'RSI_6'] > _ticker.loc[_ticker.index[i-1], 'RSI_12'] \
            and _ticker.loc[_ticker.index[i],'RSI_6'] < _ticker.loc[_ticker.index[i], 'RSI_12']:
                res.iloc[i, col_b] = -1
                
    return res

In [56]:
### 410974045 ###
def ma_1_strategy(_ticker):
    
    ## 在單一策略裡呼叫需要使用的MA
    _ticker = MA(_ticker, 5)
    _ticker = MA(_ticker, 12)
    _ticker = MA(_ticker, 25)
    
    res = _ticker.copy(deep=True)
    res['buy_point'] = 0
    col_b = res.columns.get_loc('buy_point')
    
    ## 買進： 收盤價交叉5日均收向上 且 5日均收 > 12日均收 且 12日均收 > 25日均收 \
    ## 賣出： 收盤價交叉5日均收向下 且 5日均收 < 12日均收 且 12日均收 < 25日均收
    for i in range(1, len(_ticker)):
        if _ticker.loc[_ticker.index[i-1],'Close'] < _ticker.loc[_ticker.index[i-1], 'MA_5'] \
            and _ticker.loc[_ticker.index[i],'Close'] > _ticker.loc[_ticker.index[i],'MA_5'] \
            and _ticker.loc[_ticker.index[i],'MA_5'] > _ticker.loc[_ticker.index[i],'MA_12'] \
            and _ticker.loc[_ticker.index[i],'MA_12'] > _ticker.loc[_ticker.index[i],'MA_25']:
                res.iloc[i, col_b] = 1
        elif _ticker.loc[_ticker.index[i-1],'Close'] > _ticker.loc[_ticker.index[i-1],'MA_5'] \
            and _ticker.loc[_ticker.index[i],'Close'] < _ticker.loc[_ticker.index[i],'MA_5'] \
            and _ticker.loc[_ticker.index[i],'MA_5'] < _ticker.loc[_ticker.index[i],'MA_12'] \
            and _ticker.loc[_ticker.index[i],'MA_12'] < _ticker.loc[_ticker.index[i],'MA_25']:
                res.iloc[i, col_b] = -1
                
    return res

In [57]:
### 410974045 ###
def rsi_kd_strategy(_ticker):
    ## 之前提的兩個策略的其中一個
    ## 當RSI>50且KD值向上形成黃金交叉時買進；反之RSI>80且KD向下形成死亡交叉時賣出
    
    res = _ticker.copy(deep=True)
    res['buy_point'] = 0
    col_b = res.columns.get_loc('buy_point')

    for i in range(1, len(_ticker)):
        if _ticker.loc[_ticker.index[i-1], 'RSI'] > 50 \
           and _ticker.loc[_ticker.index[i], 'K'] > _ticker.loc[_ticker.index[i], 'D'] \
           and _ticker.loc[_ticker.index[i-1], 'K'] < _ticker.loc[_ticker.index[i-1], 'D'] :
            res.iloc[i, col_b] = 1
        elif _ticker.loc[_ticker.index[i-1], 'RSI'] > 80 \
           and _ticker.loc[_ticker.index[i], 'K'] < _ticker.loc[_ticker.index[i], 'D'] \
           and _ticker.loc[_ticker.index[i-1], 'K'] > _ticker.loc[_ticker.index[i-1], 'D'] :
            res.iloc[i, col_b] = -1
            
    return res

## KD + RSI
* 多 : KD 黃金交叉，且 RSI 也大於某一數值
* 空 : KD 死亡交叉，且 RSI 也小於某一數值

In [58]:
def kd_rsi_rule(_ticker, buy_point_name='buy_point', rsi_period=14, kd_period=9, 
        rsi_buy_point=20, rsi_sell_point=80, period=5):
    
    # 複製一份資料，避免修改原始資料
    res = _ticker.copy(deep=True)
    
    # 檢查使否存在藥用的指標
    try:
        if not {'K', 'D'}.issubset(res.columns):
            res = KD(res, kd_period)
        if not {'RSI'}.issubset(res.columns):
            res = RSI(res, rsi_period)
    except Exception as e:
        print(e)
        return False
    
    # 新增一個空行，預設值為零
    res[buy_point_name] = 0
    t = res.index

    for i in range(1, len(res)):
        if res.loc[t[i], 'K'] > res.loc[t[i], 'D'] \
            and res.loc[t[i - 1], 'K'] < res.loc[t[i - 1], 'D'] \
            and res.loc[t[i], 'RSI'] > rsi_buy_point:

            res.loc[t[i], 'buy_point'] = 1
        
        elif res.loc[t[i], 'K'] < res.loc[t[i], 'D'] \
            and res.loc[t[i - 1], 'K'] > res.loc[t[i - 1], 'D'] \
            and res.loc[t[i], 'RSI'] < rsi_sell_point:

            res.loc[t[i], 'buy_point'] = -1

    return res

## BBands + EMA + RSI

* 多 : 布林帶撞破下軌後，爬回均線，且RSI低於一定值
* 空 : 布林帶撞破上軌後，跌回均線，且RSI高於一定值

In [59]:
def bbands_ema_rsi_rule(_ticker, buy_point_name='buy_point', 
        bb_fast=12, bb_slow=26, bb_period=9, rsi_period=14, 
        rsi_buy_point=40, rsi_sell_point=60, ma_period=5):
    
    # 複製一份資料，避免修改原始資料
    res = _ticker.copy(deep=True)

    # 檢查使否存在要用的指標
    try:
        if not {'bbands_middle', 'bbands_upper', 'bbands_lower'}.issubset(res.columns):
            res = BBands(res, bb_fast, bb_slow, bb_period)
        if not {'RSI'}.issubset(res.columns):
            res = RSI(res, rsi_period)
        if not {'EMA_' + str(ma_period)}.issubset(res.columns):
            res = EMA(res, ma_period)
    except Exception as e:
        print(e)
        return False

    res[buy_point_name] = 0
    t = res.index
    is_lower = False
    is_upper = False

    for i in range(1, len(res)):

        if res.loc[t[i], 'Close'] < res.loc[t[i], 'bbands_lower']:
            is_lower = True
        elif res.loc[t[i], 'Close'] > res.loc[t[i], 'bbands_middle']:
            is_lower = False
        
        if res.loc[t[i], 'Close'] > res.loc[t[i], 'bbands_upper']:
            is_upper = True
        elif res.loc[t[i], 'Close'] < res.loc[t[i], 'bbands_middle']:
            is_upper = False
        
        
        if is_lower and res.loc[t[i], 'Close'] > res.loc[t[i], 'EMA_' + str(ma_period)] \
            and res.loc[t[i], 'RSI'] < rsi_buy_point:
            
            res.loc[t[i], 'buy_point'] = 1
        
        elif is_upper and res.loc[t[i], 'Close'] < res.loc[t[i], 'EMA_' + str(ma_period)] \
            and res.loc[t[i], 'RSI'] > rsi_sell_point:
            
            res.loc[t[i], 'buy_point'] = -1

    return res

In [60]:
def rsi_macd_strategy(_ticker):
    
    ## 在單一策略裡呼叫需要使用的RSI
    _ticker = RSI_2(_ticker, 6)  
    _ticker = RSI_2(_ticker, 12)
    
    res = _ticker.copy(deep=True)
    res['buy_point'] = 0
    col_b = res.columns.get_loc('buy_point')
    
    ## 6日RSI交叉12日RSI向上時買進 \ 6日RSI交叉12日RSI向下時賣出
    for i in range(1,len(_ticker)):
        if _ticker.loc[_ticker.index[i-1],'RSI_6'] < _ticker.loc[_ticker.index[i-1], 'RSI_12'] \
            and  _ticker.loc[_ticker.index[i],'RSI_6'] > _ticker.loc[_ticker.index[i], 'RSI_12'] \
            and _ticker.loc[_ticker.index[i-1], 'macd_dif'] < _ticker.loc[_ticker.index[i-1], 'macd_dea'] \
            and _ticker.loc[_ticker.index[i], 'macd_dif'] > _ticker.loc[_ticker.index[i], 'macd_dea']:
                res.iloc[i, col_b] = 1
                
        elif _ticker.loc[_ticker.index[i-1],'RSI_6'] > _ticker.loc[_ticker.index[i-1], 'RSI_12'] \
            and _ticker.loc[_ticker.index[i],'RSI_6'] < _ticker.loc[_ticker.index[i], 'RSI_12'] \
            and _ticker.loc[_ticker.index[i-1], 'macd_dif'] > _ticker.loc[_ticker.index[i-1], 'macd_dea'] \
            and _ticker.loc[_ticker.index[i], 'macd_dif'] < _ticker.loc[_ticker.index[i], 'macd_dea']:
                res.iloc[i, col_b] = -1
                
    return res

In [61]:
def ma_rule(_ticker, buy_point_name='buy_point', period_f=5, period_s=10):
    
    # 複製一份資料，避免修改原始資料
    res = _ticker.copy(deep=True)

    # 檢查使否存在要用的指標
    try:
        if not {'MA_' + str(period_f)}.issubset(res.columns):
            res = MA(res, period_f)
        if not {'MA_' + str(period_s)}.issubset(res.columns):
            res = MA(res, period_s)
    except Exception as e:
        print(e)
        return False

    # 新增一個空行，預設值為零
    res[buy_point_name] = 0

    # 取得 index 值，方便計算使用
    t = res.index

    # 計算是否符合多空點
    for i in range(0, len(_ticker)):
        
        if res.loc[t[i], 'Close'] > res.loc[t[i], 'MA_' + str(period_f)] \
                > res.loc[t[i], 'MA_' + str(period_s)]:

            res.loc[res.index[i], 'buy_point'] = 1

        elif res.loc[t[i], 'Close'] < res.loc[t[i], 'MA_' + str(period_f)] \
                < res.loc[t[i], 'MA_' + str(period_s)]:

            res.loc[res.index[i], 'buy_point'] = -1

    return res

## MACD + RSI

* 多 : MACD 黃金交叉 + RSI 介於一定值內
* 空 : 

* 因為macd 遇到股價震盪時容易失效，所以以RSI指標去確定此時股價的趨勢不是在最低點或最高點

In [62]:
def muti_indi_rule_1(_ticker, buy_point_name='buy_point', rsi_upper_bound=80, rsi_lower_bound=20, period=5):
    
    try:
        if not {'macd_dif', 'macd_dea', 'macd'}.issubset(_ticker.columns):
            _ticker = MACD(_ticker)
        if not {'RSI'}.issubset(_ticker.columns):
            _ticker = RSI(_ticker)
    except Exception as e:
        print(e)
        return False

    res = _ticker.copy(deep=True)
    res[buy_point_name] = 0

    for i in range(1, len(_ticker)):
        if _ticker.loc[_ticker.index[i], 'macd_dif'] > _ticker.loc[_ticker.index[i], 'macd_dea'] \
                and _ticker.loc[_ticker.index[i - 1], 'macd_dea'] > _ticker.loc[_ticker.index[i - 1], 'macd_dif'] \
                and _ticker.loc[_ticker.index[i], 'RSI'] > rsi_lower_bound \
                and _ticker.loc[_ticker.index[i], 'RSI'] < rsi_upper_bound:

            res.loc[res.index[i], 'buy_point'] = 1

        # elif (_ticker.loc[_ticker.index[i], 'macd_dea'] > _ticker.loc[_ticker.index[i], 'macd_dif']) and \
        #     (_ticker.loc[_ticker.index[i - 1], 'macd_dif'] > _ticker.loc[_ticker.index[i - 1], 'macd_dea']) and \
        #     (_ticker.loc[_ticker.index[i], 'RSI'] > 50) and \
        #     (_ticker.loc[_ticker.index[i], 'macd_dif'] > 0):

        #     res.loc[res.index[i], 'buy_point'] = -1

    return res

## MACD

In [63]:
def muti_indi_rule_2(_ticker, buy_point_name='buy_point'):
    try:
        if not {'macd_dif', 'macd_dea', 'macd'}.issubset(_ticker.columns):
            _ticker = MACD(_ticker)
        if not {'RSI'}.issubset(_ticker.columns):
            _ticker = RSI(_ticker)
    except Exception as e:
        print(e)
        return False

    res = _ticker.copy(deep=True)
    res[buy_point_name] = 0

    for i in range(1, len(_ticker)):
        if _ticker.loc[_ticker.index[i], 'macd_dif'] > _ticker.loc[_ticker.index[i], 'macd_dea'] \
                and _ticker.loc[_ticker.index[i - 1], 'macd_dea'] > _ticker.loc[_ticker.index[i - 1], 'macd_dif']:

            res.loc[res.index[i], 'buy_point'] = 1

        # elif (_ticker.loc[_ticker.index[i], 'macd_dea'] > _ticker.loc[_ticker.index[i], 'macd_dif']) and \
        #     (_ticker.loc[_ticker.index[i - 1], 'macd_dif'] > _ticker.loc[_ticker.index[i - 1], 'macd_dea']) and \
        #     (_ticker.loc[_ticker.index[i], 'RSI'] > 50) and \
        #     (_ticker.loc[_ticker.index[i], 'macd_dif'] > 0):

        #     res.loc[res.index[i], 'buy_point'] = -1

    return res

## MACD

* 多 : MACD 黃金交叉，且前三日的零位面下柱體體積多於一定值
* 空 : MACD 死亡交叉，且前三日的零位面上柱體體積多於一定值

In [64]:
def muti_indi_rule_3(_ticker, buy_point_name='buy_point', period=5):
    try:
        if not {'macd_dif', 'macd_dea', 'macd'}.issubset(_ticker.columns):
            _ticker = MACD(_ticker)
    except Exception as e:
        print(e)
        return False

    res = _ticker.copy(deep=True)
    res[buy_point_name] = 0
    t = _ticker.index

    bar_size = 0

    for i in range(0, period):
        bar_size += _ticker.loc[t[i], 'macd']

    for i in range(period, len(_ticker)):

        bar_size += _ticker.loc[t[i], 'macd']
        
        if _ticker.loc[t[i], 'macd'] > 0 and _ticker.loc[t[i - 1], 'macd'] < 0 \
                and bar_size < -2:

            res.loc[res.index[i], 'buy_point'] = 1

        elif _ticker.loc[t[i], 'macd'] < 0 and _ticker.loc[t[i - 1], 'macd'] > 0 \
                and bar_size > 1:

            res.loc[res.index[i], 'buy_point'] = -1
        
        bar_size -= _ticker.loc[t[i - period], 'macd']

    return res

In [65]:
def muti_indi_rule_5(_ticker, buy_point_name='buy_point', period=5):
    
    res = _ticker.copy(deep=True)

    try:
        if not {'macd_dif', 'macd_dea', 'macd'}.issubset(_ticker.columns):
            _ticker = MACD(_ticker)
    except Exception as e:
        print(e)
        return False

    res[buy_point_name] = 0
    t = res.index


    for i in range(period, len(_ticker)):
        
        if res.loc[t[i], 'macd'] > 0 and res.loc[t[i - 1], 'macd'] < 0 \
                and res.loc[t[i], 'Close'] > res.loc[t[i], 'bbands_lower']:

            res.loc[res.index[i], 'buy_point'] = 1

        elif _ticker.loc[t[i], 'macd'] < 0 and _ticker.loc[t[i - 1], 'macd'] > 0 \
                and res.loc[t[i], 'Close'] < res.loc[t[i], 'bbands_upper']:

            res.loc[res.index[i], 'buy_point'] = -1

    return res

## MACD + 均線

In [66]:
def muti_indi_rule_6(_ticker, buy_point_name='buy_point', period=5):
    
    res = _ticker.copy(deep=True)

    try:
        if not {'macd_dif', 'macd_dea', 'macd'}.issubset(_ticker.columns):
            _ticker = MACD(_ticker)
    except Exception as e:
        print(e)
        return False

    res[buy_point_name] = 0
    t = res.index


    for i in range(period, len(_ticker)):
        
        if res.loc[t[i], 'macd'] > 0 and res.loc[t[i], 'Close'] > res.loc[t[i], 'MA_' + str(period)]:

            res.loc[res.index[i], 'buy_point'] = 1

        elif _ticker.loc[t[i], 'macd'] < 0 and _ticker.loc[t[i - 1], 'macd'] > 0 \
                and res.loc[t[i], 'Close'] < res.loc[t[i], 'bbands_upper']:

            res.loc[res.index[i], 'buy_point'] = -1

    return res

##  均線

In [67]:
def muti_indi_rule_7(_ticker, buy_point_name='buy_point', period_f=5, period_s=10):
    
    res = _ticker.copy(deep=True)

    try:
        if not {'MA_' + str(period_f)}.issubset(res.columns):
            res = MA(res, period_f)
        if not {'MA_' + str(period_s)}.issubset(res.columns):
            res = MA(res, period_s)
        
    except Exception as e:
        print(e)
        return False

    res[buy_point_name] = 0
    t = res.index


    for i in range(0, len(_ticker)):
        
        if res.loc[t[i], 'Close'] > res.loc[t[i], 'MA_' + str(period_f)] \
                and res.loc[t[i], 'MA_' + str(period_f)] > res.loc[t[i], 'MA_' + str(period_s)]:

            res.loc[res.index[i], 'buy_point'] = 1

        elif res.loc[t[i], 'Close'] < res.loc[t[i], 'MA_' + str(period_f)] \
                and res.loc[t[i], 'MA_' + str(period_f)] < res.loc[t[i], 'MA_' + str(period_s)]:

            res.loc[res.index[i], 'buy_point'] = -1

    return res

## MACD + 成交量

MACD柱狀斜率向上5天 + KD黃金交叉者(過濾成交量<500)

In [68]:
def muti_indi_rule_4(_ticker, buy_point_name='buy_point'):
    try:
        if not {'macd_dif', 'macd_dea', 'macd'}.issubset(_ticker.columns):
            _ticker = MACD(_ticker)
        if not {'VMA_5'}.issubset(_ticker.columns):
            _ticker = MA(_ticker, 5, 'Volume')
    except Exception as e:
        print(e)
        return False

    res = _ticker.copy(deep=True)
    res[buy_point_name] = 0

    for i in range(1, len(_ticker)):
        if _ticker.loc[_ticker.index[i], 'macd'] > _ticker.loc[_ticker.index[i - 1], 'macd_dea'] \
            > _ticker.loc[_ticker.index[i - 2], 'macd'] > _ticker.loc[_ticker.index[i - 3], 'macd'] \
            > _ticker.loc[_ticker.index[i - 4], 'macd'] \
            and _ticker.loc[_ticker.index[i], 'Volume'] > _ticker.loc[_ticker.index[i], 'VMA_5']:

            res.loc[res.index[i], 'buy_point'] = 1

        # elif _ticker.loc[_ticker.index[i], 'RSI'] > 80:

        #     res.loc[res.index[i], 'buy_point'] = -1

    return res

In [69]:
def muti_indi_rule_00(_ticker, buy_point_name='buy_point', period=5):
    try:
        if not {'macd_dif', 'macd_dea', 'macd'}.issubset(_ticker.columns):
            _ticker = MACD(_ticker)
    except Exception as e:
        print(e)
        return False
    
    _ticker = MA(_ticker, 6)
    _ticker = MA(_ticker, 12)
    _ticker = MA(_ticker, 25)

    res = _ticker.copy(deep=True)
    res[buy_point_name] = 0
    t = _ticker.index

    bar_size = 0

    for i in range(0, period):
        bar_size += _ticker.loc[t[i], 'macd']

    for i in range(period, len(_ticker)):

        bar_size += _ticker.loc[t[i], 'macd']
        
        if _ticker.loc[t[i], 'macd'] > 0 and _ticker.loc[t[i - 1], 'macd'] < 0 \
                and bar_size < -0.1 \
                and _ticker.loc[_ticker.index[i],'MA_6'] > _ticker.loc[_ticker.index[i],'MA_12'] :

            res.loc[res.index[i], 'buy_point'] = 1

        elif _ticker.loc[t[i], 'macd'] < 0 and _ticker.loc[t[i - 1], 'macd'] > 0 \
                and bar_size > 1 \
                and _ticker.loc[_ticker.index[i],'MA_6'] < _ticker.loc[_ticker.index[i],'MA_12'] :

            res.loc[res.index[i], 'buy_point'] = -1
        
        bar_size -= _ticker.loc[t[i - period], 'macd']

    return res

# 繪圖

## MUTI_PLOT

In [70]:
def plot_muti(_ticker, _title='', volume=True, buy_sell_point=False, 
    RSI=False, rsi_midline=False, rsi_bound=False, rsi_upper_bound=85, rsi_lower_bound=15, 
    KD=False, kd_bound=False, kd_upper_bound=85, kd_lower_bound=15, 
    MACD=False, BBands=False):

    apds = []
    _panel = 1

    # Add the buy point and sell point to plot
    if buy_sell_point:
        
        buy, sell = [np.nan], [np.nan]
        buy_b = False
        sell_b = False

        # Adjust the position to avoid signals block the lines
        for i in range(1, len(_ticker)):

            if _ticker.loc[_ticker.index[i], 'ticker_num'] > _ticker.loc[_ticker.index[i - 1], 'ticker_num']:
                buy.append(_ticker.loc[_ticker.index[i], 'Low'] * 0.97)
                buy_b = True
            else:
                buy.append(np.nan)

            if _ticker.loc[_ticker.index[i], 'ticker_num'] < _ticker.loc[_ticker.index[i - 1], 'ticker_num']:
                sell.append(_ticker.loc[_ticker.index[i], 'High'] * 1.03)
                sell_b = True
            else:
                sell.append(np.nan)

        if buy_b:
            apds += [ 
            mpf.make_addplot(buy, panel=0, type = 'scatter', color = 'blue', marker = '^', markersize = 100)
            ]

        if sell_b:
            apds += [ 
                mpf.make_addplot(sell, panel=0, type = 'scatter', color = 'darkorange', marker = 'v', markersize = 100),
            ]

    # Add the RSI indicator to plot
    if RSI:

        _panel += 1
        apds += [ 
            mpf.make_addplot(_ticker['RSI'], panel = _panel - 1, color='#009393', ylabel='RSI', secondary_y=False)
        ]

        # Add upperbound & lowerbound to plot
        if rsi_bound:
            apds += [
                mpf.make_addplot(np.full((len(_ticker)), rsi_lower_bound), 
                    panel = _panel - 1, color='#F00078', alpha=0.5, secondary_y=False),
                mpf.make_addplot(np.full((len(_ticker)), rsi_upper_bound), 
                    panel = _panel - 1, color='#F00078', alpha=0.5, secondary_y=False)
            ]

        # Add middleLine to plot
        if rsi_midline:
            apds += [mpf.make_addplot(np.full((len(_ticker)), 50), 
                    panel = _panel - 1, color='#F00078', alpha=0.5)]

    # Add the KD indicator to plot
    if KD:

        _panel += 1
        apds += [ 
            mpf.make_addplot(_ticker['K'], panel=_panel - 1, color='#0000C6', 
                ylabel='KD'),
            mpf.make_addplot(_ticker['D'], panel=_panel - 1, color='#F75000'),
            ]

        # Add upperbound & lowerbound to plot
        if kd_bound:
            apds += [
                mpf.make_addplot(np.full((len(_ticker)), kd_upper_bound), 
                    panel=_panel - 1, color='#F00078', alpha=0.5, secondary_y=False),
                mpf.make_addplot(np.full((len(_ticker)), kd_lower_bound), 
                    panel=_panel - 1, color='#F00078', alpha=0.5, secondary_y=False)
            ]

    # Add MACD indicator to plot
    if MACD:

        _panel += 1
        apds += [
            mpf.make_addplot(_ticker['macd'], type='bar', ylabel='MACD', width=0.7, panel=_panel - 1,
                color='dimgray', alpha=1, secondary_y=False),
            mpf.make_addplot(_ticker['macd_dif'], panel=_panel - 1, color='fuchsia', secondary_y=True),
            mpf.make_addplot(_ticker['macd_dea'], panel=_panel - 1, color='b', secondary_y=True)
        ]

    if volume:
        _panel += 1

    if BBands:
        apds += [
            mpf.make_addplot(_ticker['bbands_middle'], panel=0, color='#9F4D95', alpha=1, width=1, secondary_y=False),
            mpf.make_addplot(_ticker['bbands_upper'], panel=0, color='black', alpha=0.5, width=0.8, secondary_y=False),
            mpf.make_addplot(_ticker['bbands_lower'], panel=0, color='black', alpha=0.5, width=0.8, secondary_y=False),
           
        ]

    # 設置k線圖顏色
    my_color = mpf.make_marketcolors(up='red',  # 上漲
                                     down='green',  # 下跌
                                     edge='i',  # 隱藏K線邊緣
                                     volume='in',  # 成交量用同樣的顏色
                                     inherit=True)

    my_style = mpf.make_mpf_style(gridaxis='both',  # 設置網格
                                  gridstyle='-.',
                                  y_on_right=True,
                                  marketcolors=my_color)

    mpf.plot(_ticker, 
        type='candle',
        num_panels=_panel,
        main_panel=0, 
        volume=volume, 
        volume_panel=_panel - 1,
        addplot = apds,
        # figratio=(18, 15), 
        figscale=1.5,
        # panel_ratios=(6, 3, 3, 2), 
        style=my_style,
        title=_title, 
        show_nontrading=False,
        tight_layout=False
        )
    return

# 資料讀取

In [71]:
def read_ticker_v1(_ticker_code):
    
    try:
        res = pd.read_csv('C:\Coding\\43_School_S3\investment\\0_final_project\data\{}.csv'.format(str(_ticker_code)), 
            index_col=0, parse_dates=True)
    except Exception as e:
        print(e)
        return False

    return res

In [72]:
def read_ticker_v2(_ticker_code):
    
    try:
        import os
        
        # get the current working path of this file
        path = os.getcwd()

        # get the data as a dataframe 
        res = pd.read_csv(path + '\\data\{}'.format(str(_ticker_code)))

        # change the name of columns
        res.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

        # set the column "Date" to index
        res = res.set_index('Date')

        # set index column("Date") from string to datetime.datetime
        res.index = pd.to_datetime(res.index)

        # sort index column("Date") chronologically
        res = res.sort_index()

    except Exception as e:
        print(e)
        return False

    return res

In [73]:
def get_ticker_list():
    
    import os

    # get the current working path of this file
    path = os.getcwd()

    # get all files in this path
    list = os.listdir(path + '\\data')
    
    # filter out the stock file (.csv)
    res = [file for file in list if '.csv' in file]

    return res

# 資料回測

## V4

* 當日盤後進行買賣
* 最少可以買一股
* 遇到做空點會賣股票

In [74]:
def back_track_v4(_ticker, funds=1_000_000, stop_loss_percent=0.96):

    # 止損價
    stop_loss = 0
    # 取得資料列名稱的串列，方便後面使用
    t = _ticker.index
    # 複製原資料
    res = _ticker.copy(deep=True)

    # 增加新的欄位，並設定預設值
    res['ticker_num'] = 0
    res['income'] = 0
    res['expenditure'] = 0
    res['account'] = funds


    for i in range(1, len(_ticker)):
        
        # 做多點
        if i != len(_ticker) - 1 and _ticker.loc[t[i], 'buy_point'] == 1 \
                and res.loc[t[i - 1], 'ticker_num'] == 0 :
            
            res.loc[t[i], 'ticker_num'] \
                = math.floor(res.loc[t[i - 1], 'account'] / (_ticker.loc[t[i], 'Close'] * 1.001425))
            res.loc[t[i], 'expenditure'] \
                = -res.loc[t[i], 'ticker_num'] * (_ticker.loc[t[i], 'Close'] * 1.001425)
            res.loc[t[i], 'account'] = res.loc[t[i - 1], 'account'] + res.loc[t[i], 'expenditure']
            # 計算目前止損停利價格
            stop_loss = _ticker.loc[t[i], 'Close'] * stop_loss_percent
        
        # 賣出點 # 移動停損停利點賣出 # 最後一天無條件賣出
        elif _ticker.loc[t[i], 'buy_point'] == -1 and res.loc[t[i - 1], 'ticker_num'] > 0 \
                or res.loc[t[i - 1], 'ticker_num'] > 0 and _ticker.loc[t[i], 'Close'] < stop_loss\
                or i == len(_ticker) - 1 and res.loc[_ticker.index[i - 1], 'ticker_num'] > 0:
            
            res.loc[t[i], 'income'] \
                = res.loc[t[i - 1], 'ticker_num'] * (_ticker.loc[t[i], 'Close'] * 0.995575)
            res.loc[t[i], 'ticker_num'] = 0
            res.loc[t[i], 'account'] = res.loc[t[i - 1], 'account'] + res.loc[t[i], 'income']
        
        else :
            # 更新此日持有股數
            res.loc[t[i], 'ticker_num'] = res.loc[t[i - 1], 'ticker_num']
            # 更新此日帳戶餘額
            res.loc[t[i], 'account'] = res.loc[t[i - 1], 'account']
            # update the stop_loss price
            if _ticker.loc[t[i], 'Close'] * stop_loss_percent > stop_loss:
                stop_loss = _ticker.loc[t[i], 'Close'] * stop_loss_percent

    return res

## V5

* 只會在停損停利點賣股票

In [75]:
def back_track_v5(_ticker, funds=1_000_000, stop_loss_percent=0.96):

    stop_loss = 0 # 止損價
    t = _ticker.index

    res = _ticker.copy(deep=True)

    res['ticker_num'] = 0
    res['income'] = 0
    res['expenditure'] = 0
    res['account'] = funds


    for i in range(1, len(_ticker)):
        
        # 做多點
        if i != len(_ticker) - 1 and _ticker.loc[t[i], 'buy_point'] == 1 \
                and res.loc[t[i - 1], 'ticker_num'] == 0 :
            
            res.loc[t[i], 'ticker_num'] \
                = math.floor(res.loc[t[i - 1], 'account'] / (_ticker.loc[t[i], 'Close'] * 1.001425))
            res.loc[t[i], 'expenditure'] \
                = -res.loc[t[i], 'ticker_num'] * (_ticker.loc[t[i], 'Close'] * 1.001425)
            res.loc[t[i], 'account'] = res.loc[t[i - 1], 'account'] + res.loc[t[i], 'expenditure']
            # 計算目前止損停利價格
            stop_loss = _ticker.loc[t[i], 'Close'] * stop_loss_percent
        
        # 移動停損停利點賣出 # 最後一天無條件賣出
        elif res.loc[t[i - 1], 'ticker_num'] > 0 and _ticker.loc[t[i], 'Close'] < stop_loss\
                or i == len(_ticker) - 1 and res.loc[_ticker.index[i - 1], 'ticker_num'] > 0:
            
            res.loc[t[i], 'income'] \
                = res.loc[t[i - 1], 'ticker_num'] * (_ticker.loc[t[i], 'Close'] * 0.995575)
            res.loc[t[i], 'ticker_num'] = 0
            res.loc[t[i], 'account'] = res.loc[t[i - 1], 'account'] + res.loc[t[i], 'income']
        
        else :
            # 更新此日持有股數
            res.loc[t[i], 'ticker_num'] = res.loc[t[i - 1], 'ticker_num']
            # 更新此日帳戶餘額
            res.loc[t[i], 'account'] = res.loc[t[i - 1], 'account']
            # update the stop_loss price
            if _ticker.loc[t[i], 'Close'] * stop_loss_percent > stop_loss:
                stop_loss = _ticker.loc[t[i], 'Close'] * stop_loss_percent

    return res

# 資料數據整理

## 交易點數據

In [76]:
def get_trade_info_v2(_ticker):

    #紀錄總交易資訊
    detail = []
    #紀錄單列交易資訊
    temp_detail = []
    
    #買入價格，方便賣出時計算報酬
    buy_price = 0
    # 紀錄起始資金，方便計算總報酬
    funds = _ticker.loc[_ticker.index[0], 'account']

    for i in range(1, len(_ticker)):

        # buy info # 交易資訊
        if _ticker.loc[_ticker.index[i], 'ticker_num'] \
            > _ticker.loc[_ticker.index[i - 1], 'ticker_num']:
                    
            # 交易日期
            temp_detail.append(_ticker.index[i])
            # 交易數量(股)
            temp_detail.append(_ticker.loc[_ticker.index[i], 'ticker_num'] \
                - _ticker.loc[_ticker.index[i - 1], 'ticker_num'])
            # 交易價格
            temp_detail.append(_ticker.loc[_ticker.index[i], 'Close'])
            # 交易價格(含稅)
            temp_detail.append(_ticker.loc[_ticker.index[i], 'Close'] * 1.001425)
            # 總手續費(手續費加交易稅)
            temp_detail.append(_ticker.loc[_ticker.index[i], 'Close'] \
                * 0.001425 * _ticker.loc[_ticker.index[i], 'ticker_num'])
            # 總花費
            temp_detail.append(_ticker.loc[_ticker.index[i], 'expenditure'])
            # 此次交易報酬，買入時無資料
            temp_detail.append(np.nan)
            # 此次交易報酬率，買入時無資料
            temp_detail.append(np.nan)
            # 目前總交易報酬，買入時無資料
            temp_detail.append(np.nan)
            # 目前總交易報酬率，買入時無資料
            temp_detail.append(np.nan)
            # 目前帳戶餘額
            temp_detail.append(_ticker.loc[_ticker.index[i], 'account'])
            # 更新買價，方便下次賣出時計算
            buy_price = _ticker.loc[_ticker.index[i], 'expenditure']

            # 將資料推入串列
            detail.append(temp_detail)
            # 進入下一列
            temp_detail = []

        # sell info
        elif _ticker.loc[_ticker.index[i], 'ticker_num'] \
            < _ticker.loc[_ticker.index[i - 1], 'ticker_num']:
            
            # 交易日期
            temp_detail.append(_ticker.index[i])
            # 交易數量(股)
            temp_detail.append(_ticker.loc[_ticker.index[i], 'ticker_num'] \
                - _ticker.loc[_ticker.index[i - 1], 'ticker_num'])
            # 交易價格
            temp_detail.append(_ticker.loc[_ticker.index[i], 'Close'])
            # 交易價格(含稅)
            temp_detail.append(_ticker.loc[_ticker.index[i], 'Close'] * 0.995575)
            # 總手續費(手續費加交易稅)
            temp_detail.append(_ticker.loc[_ticker.index[i], 'Close'] \
                * 0.004425 *  _ticker.loc[_ticker.index[i - 1], 'ticker_num'])
            # 總收入 
            temp_detail.append(_ticker.loc[_ticker.index[i], 'income'])
            # 此次交易報酬
            temp_detail.append(_ticker.loc[_ticker.index[i], 'income'] + buy_price)
            # 此次交易報酬率
            temp_detail.append((_ticker.loc[_ticker.index[i], 'income'] + buy_price) / (-buy_price) * 100)
            # 目前總交易報酬
            temp_detail.append(_ticker.loc[_ticker.index[i], 'account'] - funds)
            # 目前總交易報酬率
            temp_detail.append((_ticker.loc[_ticker.index[i], 'account'] - funds) / funds * 100)
            # 目前帳戶餘額
            temp_detail.append(_ticker.loc[_ticker.index[i], 'account'])

            # 將資料推入串列
            detail.append(temp_detail)
            # 進入下一列
            temp_detail = []
            
    # 將串列改成 dataframe 後回傳，並加上各行名稱
    return pd.DataFrame(detail, columns=['Date', 'ticker_num', 'price', 'price(with_tax)', 'tax', 
        'spend', 'profit', 'profit_rate', 'profit_sum', 'profit_sum_rate', 'account'])

## 交易結論數據

In [77]:
def get_result(_df):
    # 回傳 買賣次數  贏 輸 勝率 報酬率
    # ['Trades', 'Win', 'Loss', 'Win_Rate', 'Profit']
    
    # 計算勝利的次數
    win = 0
    #計算敗的次數
    loss = 0
    
    # 報酬大於零是為勝利，反之為敗
    for i in range(1, len(_df), 2):
        if _df.loc[_df.index[i], 'profit_rate'] > 0:
            win += 1
        else :
            loss += 1

    # 如果沒有任何交易，例外處理
    if win + loss == 0:
        return [win + loss, win, loss, 0, 0]

    # 回傳資料
    return [win + loss, win, loss, win / (win + loss), _df.loc[_df.index[len(_df) - 1], 'profit_sum_rate']]

# 測試

In [78]:
def test(ticker, strategy, funds=500_000, stop_loss_percent=0.96, plot=False, 
        period=5, rsi_buy_point=50, rsi_sell_point=50):
    

    # 標出交易策略多空點
    ticker = strategy(ticker, period=period, rsi_buy_point=rsi_buy_point, rsi_sell_point=rsi_sell_point)

    # 數據回測
    ticker = back_track_v4(ticker, funds=funds, stop_loss_percent=stop_loss_percent) # 起始資金為五十萬

    # 繪圖
    if plot:
        dff = ticker.loc['2020-06':'2020-10', ]
        plot_muti(dff, _title='', volume=True, buy_sell_point=True, 
            RSI=True, rsi_midline=True, rsi_bound=True, rsi_upper_bound=85, rsi_lower_bound=15, 
            KD=True, kd_bound=True, kd_upper_bound=85, kd_lower_bound=15, 
            MACD=True, BBands=True)

    # 計算交易資訊
    result = get_trade_info_v2(ticker)

    # 計算投資報酬率
    profit = get_result(result)

    return profit

In [79]:
data = '2371'
# 讀取資料
ticker = read_ticker_v2(data + '.csv')

a = test(ticker, kd_rsi_rule, funds=500_000, 
    stop_loss_percent=0.96, plot=False, period=5, rsi_buy_point=20, rsi_sell_point=80)

print(a)

[120, 47, 73, 0.39166666666666666, 220.8708508059491]


# 實作

## 單支股票，單個策略

In [84]:
data = '0050'
# 讀取資料
ticker = read_ticker_v2(data + '.csv')

# 加入指標
ticker = MA(ticker, 5)
# ticker = MA(ticker, 10)
# ticker = MA(ticker, 20)
ticker = EMA(ticker, 5)
# ticker = EMA(ticker, 20)
# ticker = variation(ticker)
# ticker = RSI(ticker, 13)
ticker = RSI(ticker)
# ticker = KD(ticker, 6)
ticker = KD(ticker)
ticker = MACD(ticker)
ticker = BBands(ticker)

# 刪除前幾筆存在部分指標不足的資料
ticker = ticker.iloc[20:, ]

# 標出交易策略多空點
# ticker = kd_rsi_rule(ticker, rsi_buy_point=60, rsi_sell_point=40)
# ticker = kd_rsi_rule(ticker, rsi_buy_point=20, rsi_sell_point=80)
# ticker = bbands_ema_rsi_rule(ticker, rsi_buy_point=35, rsi_sell_point=65)
ticker = ma_rule(ticker, period_f=4, period_s=8)

# 數據回測
ticker = back_track_v5(ticker, 500_000, stop_loss_percent=0.96) # 起始資金為五十萬

# 繪圖
# dff = ticker.loc['2021-01':'2021-04', ]
# plot_muti(dff, _title='', volume=True, buy_sell_point=True, 
#     RSI=True, rsi_midline=True, rsi_bound=True, rsi_upper_bound=85, rsi_lower_bound=15, 
#     KD=True, kd_bound=True, kd_upper_bound=85, kd_lower_bound=15, 
#     MACD=True, BBands=True)

# 計算交易資訊
result = get_trade_info_v2(ticker)

# 計算投資報酬率
profit = get_result(result)
print(profit)

# 資料儲存
ticker.to_csv(data + '_info.csv')
result.to_csv(data + '_result.csv')

[107, 39, 68, 0.3644859813084112, 30.437842716099787]


## 多支股票結果比較

In [81]:
# 取得資料名稱串列
ticker_list = get_ticker_list()

# 回傳資料
res = []
# 總報酬
profit_sum = 0

for data in ticker_list:

    print(data)

    # 讀取資料
    ticker = read_ticker_v2(data)

    # calculate the indicator
    ticker = MA(ticker, 5)
    ticker = MA(ticker, 5, 'Volume')
    ticker = EMA(ticker, 5)
    ticker = RSI(ticker, 13)
    ticker = KD(ticker, 6)
    ticker = MACD(ticker)
    # ticker = BBands(ticker)
    ticker = BBands(ticker, period=15, std_num=0.8)

    # 刪除前幾筆存在部分指標不足的資料
    ticker = ticker.iloc[20:, ]

    # 標出交易策略多空點
    # ticker = kd_rsi_rule(ticker, rsi_buy_point=30, rsi_sell_point=70)
    # ticker = bbands_ema_rsi_rule(ticker, rsi_buy_point=50, rsi_sell_point=70)
    # ticker = muti_indi_rule_5(ticker)
    # ticker = muti_indi_rule_7(ticker, period_f=4, period_s=8)
    ticker = ma_rule(ticker, period_f=4, period_s=8)
    # ticker = rsi_macd_strategy(ticker)

    # 數據回測
    # ticker = back_track_v4(ticker, 500_000, stop_loss_percent=0.96)
    ticker = back_track_v5(ticker, 500_000, stop_loss_percent=0.96)

    # 繪圖
    # dff = ticker.loc['2019-01':'2019-12', ]
    # plot_muti(dff, _title=data, volume=True, buy_sell_point=True, 
    #     RSI=True, rsi_midline=False, rsi_bound=True, rsi_upper_bound=60, rsi_lower_bound=40, 
    #     KD=False, kd_bound=False, kd_upper_bound=85, kd_lower_bound=15, 
    #     MACD=True, BBands=True)

    # 計算交易資訊
    result = get_trade_info_v2(ticker)

    # 計算投資報酬率
    profit = get_result(result)

    # 將該支股票之報酬率加總
    profit_sum += profit[len(profit) - 1]

    # 將當前資料加入新的一列
    res += [profit]
    
# 將陣列資料轉乘 dataframe
res = pd.DataFrame(res)

# 加入各列資料名稱 
res.set_axis(ticker_list, axis='rows', inplace=True)

# 加入各行數據名稱
res.set_axis(['Trades', 'Win', 'Loss', 'Win_Rate', 'Profit'], axis='columns', inplace=True)

# 印出總結果
print(res)

# 印出平均報酬率
print('平均報酬率 :', profit_sum / len(ticker_list))

0050.csv
1101.csv
1605.csv
2208.csv
2301.csv
2303.csv
2308.csv
2317.csv
2330.csv
2352.csv
2356.csv
2357.csv
2371.csv
2377.csv
2409.csv
2449.csv
2454.csv
2515.csv
2609.csv
2633.csv
2886.csv
2892.csv
3231.csv
3481.csv
5351.csv
6206.csv
          Trades  Win  Loss  Win_Rate       Profit
0050.csv     107   39    68  0.364486    30.437843
1101.csv      37   14    23  0.378378    -0.943329
1605.csv      61   28    33  0.459016   190.110988
2208.csv      60   17    43  0.283333   -17.280056
2301.csv      50   18    32  0.360000    -7.268040
2303.csv      30   15    15  0.500000   325.550148
2308.csv      29   13    16  0.448276    26.769213
2317.csv      26    8    18  0.307692    29.361851
2330.csv      27   10    17  0.370370    66.687397
2352.csv      55   27    28  0.490909    22.004913
2356.csv      43   16    27  0.372093   -37.277850
2357.csv      42   17    25  0.404762    13.611648
2371.csv      75   32    43  0.426667   518.935369
2377.csv      22   11    11  0.500000    36.889810
2

## 兩種回測方法比較

In [82]:
ticker_list = get_ticker_list()

res = []
res2 = []
# 總報酬
profit_sum = 0
profit_sum2 = 0

for data in ticker_list:

    print(data)

    ticker = read_ticker_v2(data)

    # calculate the indicator
    ticker = MA(ticker, 5)
    ticker = MA(ticker, 10)
    ticker = MA(ticker, 5, 'Volume')
    ticker = EMA(ticker, 5)
    ticker = RSI(ticker, 13)
    ticker = KD(ticker, 6)
    ticker = MACD(ticker)
    # ticker = BBands(ticker, std_num=2)
    ticker = BBands(ticker)
    # ticker2 = BBands(ticker, period=12, std_num=1.8)

    # 刪除前幾筆存在部分指標不足的資料
    ticker = ticker.iloc[20:, ]

    # 標出交易策略多空點
    # ticker = kd_rsi_rule(ticker, rsi_buy_point=60, rsi_sell_point=40)
    # ticker = kd_rsi_rule(ticker, rsi_buy_point=30, rsi_sell_point=70)
    # ticker2 = kd_rsi_rule(ticker, rsi_buy_point=35, rsi_sell_point=65)
    # ticker2 = bbands_ema_rsi_rule(ticker, rsi_buy_point=50, rsi_sell_point=70)
    ticker = muti_indi_rule_7(ticker, period_f=4, period_s=8)
    # ticker2 = rsi_macd_strategy(ticker)
    ticker2 = ma_1_strategy(ticker)
    # ticker2 = muti_indi_rule_00(ticker)

    # 數據回測
    ticker = back_track_v5(ticker, 500_000, stop_loss_percent=0.96)
    ticker2 = back_track_v5(ticker2, 500_000, stop_loss_percent=0.96)


    # dff = ticker.loc['2020-01':'2020-12', ]
    # plot_muti(dff, _title=data, volume=True, buy_sell_point=True, 
    #     RSI=True, rsi_midline=False, rsi_bound=True, rsi_upper_bound=60, rsi_lower_bound=40, 
    #     KD=True, kd_bound=False, kd_upper_bound=85, kd_lower_bound=15, 
    #     MACD=True, BBands=True)

    # 計算交易資訊
    result = get_trade_info_v2(ticker)
    result2 = get_trade_info_v2(ticker2)

    # 計算投資報酬率
    profit = get_result(result)
    profit2 = get_result(result2)

    # 將該支股票之報酬率加總
    profit_sum += profit[len(profit) - 1]
    profit_sum2 += profit2[len(profit2) - 1]

    # 將當前資料加入新的一列
    res += [profit]
    res2 += [profit2]
    
# 將陣列資料轉乘 dataframe
res = pd.DataFrame(res)
res2 = pd.DataFrame(res2)

# 加入各列資料名稱 
res.set_axis(ticker_list, axis='rows', inplace=True)
res2.set_axis(ticker_list, axis='rows', inplace=True)

# 加入各行數據名稱
res.set_axis(['Trades', 'Win', 'Loss', 'Win_Rate', 'Profit'], axis='columns', inplace=True)
res2.set_axis(['Trades', 'Win', 'Loss', 'Win_Rate', 'Profit'], axis='columns', inplace=True)

print(res)
# 印出平均報酬率
print('平均報酬率 :', profit_sum / len(ticker_list))
print(res2)
# 印出平均報酬率
print('平均報酬率 :', profit_sum2 / len(ticker_list))

0050.csv
1101.csv
1605.csv
2208.csv
2301.csv
2303.csv
2308.csv
2317.csv
2330.csv
2352.csv
2356.csv
2357.csv
2371.csv
2377.csv
2409.csv
2449.csv
2454.csv
2515.csv
2609.csv
2633.csv
2886.csv
2892.csv
3231.csv
3481.csv
5351.csv
6206.csv
          Trades  Win  Loss  Win_Rate       Profit
0050.csv     107   39    68  0.364486    30.437843
1101.csv      37   14    23  0.378378    -0.943329
1605.csv      61   28    33  0.459016   190.110988
2208.csv      60   17    43  0.283333   -17.280056
2301.csv      50   18    32  0.360000    -7.268040
2303.csv      30   15    15  0.500000   325.550148
2308.csv      29   13    16  0.448276    26.769213
2317.csv      26    8    18  0.307692    29.361851
2330.csv      27   10    17  0.370370    66.687397
2352.csv      55   27    28  0.490909    22.004913
2356.csv      43   16    27  0.372093   -37.277850
2357.csv      42   17    25  0.404762    13.611648
2371.csv      75   32    43  0.426667   518.935369
2377.csv      22   11    11  0.500000    36.889810
2